In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

encoder = Encoder(input_dim=10, hidden_dims=[10, 16, 32, 16], feat_dim=4)
proj_head = Proj(input_dim=16, hidden_dims=[10, 16], feat_dim=4)
classification_model = ClassificationHead(input_dim=16, num_classes=6)

# load model
checkpoint = torch.load('') # model path
encoder.load_state_dict(checkpoint['encoder'])
proj_head.load_state_dict(checkpoint['projector'])
classification_model.load_state_dict(checkpoint['cls'])

encoder.to(device)
proj_head.to(device)
classification_model.to(device)

In [ ]:
def generate_normalized_embeddings(samples):
    if isinstance(samples, np.ndarray):
        samples = torch.tensor(samples, dtype=torch.float32)
        
    embeddings = encoder(samples)

    # normalize embeddings
    normalizer = lambda x: x / (torch.norm(x, p=2, dim=-1, keepdim=True) + 1e-10)
    normalized_embeddings = normalizer(embeddings)

    return normalized_embeddings.detach().cpu().numpy()

In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

def knn_score(train_set, test_set, k):
    nbrs = NearestNeighbors(n_neighbors=k, algorithm='auto').fit(train_set)
    distances, indices = nbrs.kneighbors(test_set)
    scores = distances[:, -1]

    return scores.tolist()
